In [1]:
# Calculate the abnormal returns
df['Abnormal Return'] = df['Return'] - df['Market Return']

# Get the ticker with the highest abnormal return
highest_ticker = df.loc[df['Abnormal Return'].idxmax(), 'Ticker']

# Get the second highest abnormal return
second_highest_ticker = df.loc[df['Abnormal Return'].sort_values(ascending=False).index[1], 'Ticker']

print("Ticker with the highest abnormal return:", highest_ticker)
print("Ticker with the second highest abnormal return:", second_highest_ticker)


NameError: name 'df' is not defined

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import yfinance as yf

# Define the function to calculate the abnormal returns
def calculate_abnormal_return(ticker, benchmark_ticker, start_date, end_date):
    # Download the stock price data
    stock_data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    benchmark_data = yf.download(benchmark_ticker, start=start_date, end=end_date, progress=False)
    
    # Extract the adjusted close prices
    stock_prices = stock_data['Adj Close']
    benchmark_prices = benchmark_data['Adj Close']
    
    # Calculate the daily returns
    stock_returns = stock_prices.pct_change()
    benchmark_returns = benchmark_prices.pct_change()
    
    # Merge the returns into a single DataFrame
    merged = pd.concat([stock_returns, benchmark_returns], axis=1)
    merged.columns = [ticker, benchmark_ticker]
    
    # Calculate the regression coefficients
    X = merged[benchmark_ticker]
    X = sm.add_constant(X)
    y = merged[ticker]
    model = sm.OLS(y, X, missing='drop')
    results = model.fit()
    alpha, beta = results.params
    
    # Calculate the predicted returns
    predicted_returns = alpha + beta * merged[benchmark_ticker]
    
    # Calculate the abnormal returns
    abnormal_returns = merged[ticker] - predicted_returns
    
    return abnormal_returns


# Define the function to get the abnormal returns for a list of tickers
def get_abnormal_returns(tickers, benchmark_ticker, start_date, end_date):
    abnormal_returns = pd.DataFrame()
    for ticker in tickers:
        try:
            abnormal_returns[ticker] = calculate_abnormal_return(ticker, benchmark_ticker, start_date, end_date)
        except Exception as e:
            print(f"{ticker}: {str(e)}")
    return abnormal_returns


# Define the function to get the top n abnormal returns
def get_top_n_abnormal_returns(abnormal_returns, n):
    returns_flat = abnormal_returns.values.flatten()
    returns_flat_sorted = np.sort(returns_flat)
    top_n_flat = returns_flat_sorted[-n:]
    top_n = np.reshape(top_n_flat, (-1, abnormal_returns.shape[1]))
    top_n = pd.DataFrame(top_n, columns=abnormal_returns.columns)
    return top_n


# Define the parameters
tickers = ['AAPL', 'AMZN', 'MSFT', 'TGT', 'WMT']
benchmark_ticker = 'SPY'
start_date = '2019-01-01'
end_date = '2020-12-31'

# Get the abnormal returns
abnormal_returns = get_abnormal_returns(tickers, benchmark_ticker, start_date, end_date)

# Get the second highest abnormal return
top_2 = get_top_n_abnormal_returns(abnormal_returns, 2)
second_highest = top_2.iloc[-2,:].name

print(f"The second highest abnormal return is for {second_highest}.")


ModuleNotFoundError: No module named 'scipy'